In [19]:
import pandas as pd

# 1.LOAD DATA

In [20]:
files = {
    "bao_chau": "../data/raw/bao_chau_data.csv",
    "duy_thai": "../data/raw/duy_thai_data.csv",
    "minh_huy": "../data/raw/minh_huy_data.csv",
    "quoc_trung": "../data/raw/quoc_trung_data.csv"
}

def read_csv(path):
    return pd.read_csv(path, sep = None, engine='python')

dataframes = {name: read_csv(path) for name, path in files.items()}

for name, df in dataframes.items():
    print(name)
    print(df.info())

bao_chau
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ﻿name        1799 non-null   object
 1   description  1796 non-null   object
 2   brand        1799 non-null   object
 3   category     1799 non-null   object
dtypes: object(4)
memory usage: 56.3+ KB
None
duy_thai
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ﻿product_name  1746 non-null   object
 1   describe       1746 non-null   object
 2   brand          1746 non-null   object
 3   category       1746 non-null   object
dtypes: object(4)
memory usage: 54.7+ KB
None
minh_huy
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1947 entries, 0 to 1946
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------     

In [21]:
def normalize_product_df(df):
    # 1. Delete BOM characters
    df.columns = (
        df.columns
        .str.replace("\ufeff", "", regex=False)  
        .str.replace("﻿", "", regex=False)        
        .str.strip()
        .str.lower()
    )
    # 2. Rename columns to standard names
    col_map = {}
    for col in df.columns:
        if col in ["name", "product_name", "product name"]:
            col_map[col] = "product_name"
        elif col in ["describe", "description"]:
            col_map[col] = "description"
    df = df.rename(columns=col_map)
    # 3. Drop unnecessary columns
    if "id" in df.columns:
        df = df.drop(columns=["id"])
    # 4. Ensure all required columns are present
    for col in ["product_name", "description", "brand", "category"]:
        if col not in df.columns:
            df[col] = None

    return df[["product_name", "description", "brand", "category"]]

In [22]:
EXPECTED_COLS = ["product_name", "description", "brand", "category"]

dfs_clean = {}

for name, df in dataframes.items():
    clean_df = normalize_product_df(df)
    dfs_clean[name] = clean_df

    print(f"\n=== {name} (after normalize) ===")
    clean_df.info()


=== bao_chau (after normalize) ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  1799 non-null   object
 1   description   1796 non-null   object
 2   brand         1799 non-null   object
 3   category      1799 non-null   object
dtypes: object(4)
memory usage: 56.3+ KB

=== duy_thai (after normalize) ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  1746 non-null   object
 1   description   1746 non-null   object
 2   brand         1746 non-null   object
 3   category      1746 non-null   object
dtypes: object(4)
memory usage: 54.7+ KB

=== minh_huy (after normalize) ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1947 entries, 0 to 1946
Data columns (total 

In [23]:
df = pd.concat(dfs_clean.values(), ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7289 entries, 0 to 7288
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  7289 non-null   object
 1   description   7286 non-null   object
 2   brand         7285 non-null   object
 3   category      7289 non-null   object
dtypes: object(4)
memory usage: 227.9+ KB


# 2. XỬ LÝ MISSING VALUES

In [24]:
df.isnull().sum()

product_name    0
description     3
brand           4
category        0
dtype: int64

In [25]:
num_null_brand = df["brand"].isna().sum()
ratio = num_null_brand / len(df) * 100
print(f"Tỷ lệ thiếu brand: {ratio:.2f}%")

Tỷ lệ thiếu brand: 0.05%


Hướng xử lý:
- `description`: Thay thế giá trị thiếu bằng nội dung từ cột `product_name`. Do số lượng thiếu ít và tên sản phẩm thường chứa các từ khóa quan trọng nhất
- `brand`: Điền giá trị `"Unknown"` để tạo thành một nhóm riêng biệt

In [26]:
df['description'] = df['description'].fillna(df['product_name'])
df['brand'] = df['brand'].fillna('Unknown')

In [27]:
#KIỂM TRA LẠI
print("Số lượng missing values sau khi xử lý:")
print(df.isnull().sum())

print(f"\nKích thước dữ liệu hiện tại: {df.shape}")

Số lượng missing values sau khi xử lý:
product_name    0
description     0
brand           0
category        0
dtype: int64

Kích thước dữ liệu hiện tại: (7289, 4)


# 3. XỬ LÝ DUPLICATES

In [28]:
dup_mask = df.duplicated(subset=["product_name", "brand", "category"], keep=False)

dup_by_category = (
    df[dup_mask]
    .groupby("category")
    .size()
    .reset_index(name="num_duplicates")
    .sort_values("num_duplicates", ascending=False)
)

print(f"Tổng số dòng trùng: {dup_mask.sum()}")
print(f"Tỉ lệ dòng trùng: {dup_mask.sum() / len(df) * 100:.2f}%")
dup_by_category

Tổng số dòng trùng: 22
Tỉ lệ dòng trùng: 0.30%


,category,num_duplicates
1,Nhà cửa & Đời sống,16
2,Thể thao & Du lịch,4
0,Mỹ phẩm & Làm đẹp,2


Hướng giải quyết:
- Do tỷ lệ trùng lặp nhỏ, nên lựa chọn loại bỏ các dòng trùng lặp để dữ liệu sạch hơn

In [29]:
df_cleaned = df.drop_duplicates(
    subset=['product_name', 'brand', 'category'],
    keep='first'
).copy()

print(f"Tổng số dòng sau khi loại duplicates: {len(df_cleaned)}")
print(f"Đã loại bỏ: {len(df) - len(df_cleaned)} dòng")

Tổng số dòng sau khi loại duplicates: 7278
Đã loại bỏ: 11 dòng


# 4. CHUẨN HÓA TEXT TIẾNG VIỆT

In [30]:
import re
from bs4 import BeautifulSoup

In [31]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # 1. Xử lý HTML tags
    try:
        soup = BeautifulSoup(text, "html.parser")
        for data in soup(['style', 'script', 'head', 'title', 'meta', '[document]']):
            data.decompose()
        text = soup.get_text(" ")
    except:
        pass    
    # 2. Chuyển thành chữ thường
    text = text.lower()
    # 3. Loại bỏ URL và email
    text = re.sub(r"http\S+|www\S+|\S+@\S+", " ", text)
    # 4. Loại bỏ các ký tự xuống dòng, tab đặc biệt (\n, \t, \r, \v)
    text = re.sub(r"[\n\t\r\v]+", " ", text)
    # 5. Chuẩn hóa các từ nối dài
    text = re.sub(r'(\w)\1{2,}', r'\1', text)
    # 6. Loại bỏ ký tự đặc biệt (giữ tiếng Việt + số)
    text = re.sub(
        r"[^0-9a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ\s]",
        " ",
        text
    )
    # 7. Loại bỏ số điện thoại, mã số...
    text = re.sub(r'\d{10,}', '', text)
    # 8. Loại bỏ khoảng trắng thừa
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [32]:
for col in ["product_name", "description"]:
    df_cleaned.loc[:, col] = df_cleaned[col].apply(clean_text)
df_cleaned.head()

,product_name,description,brand,category
0,đầm tiểu thư phối cổ sang chảnh thiết kế đi là...,thông tin người mặc size s s ả n ph ẩ m vòng e...,OEM,Thời trang nữ
1,đầm maxi cổ vuông thắt nơ phối viền sang trọng,thông tin người mặc size s s ả n ph ẩ m vòng e...,OEM,Thời trang nữ
2,đầm linen dáng suông cỏ v có túi 2 bên sườn kè...,shop chuyên cung cấp quần âu áo sơ mi áo kiểu ...,ARCTIC HUNTER,Thời trang nữ
3,đầm suông nữ thiết kế rút eo tiện lợi có tui 2...,shop chuyên cung cấp quần âu áo sơ mi áo kiểu ...,ARCTIC HUNTER,Thời trang nữ
4,đầm sơ mi suông dáng xòe thiết kế tay sắn phối...,nội dung sản phẩm thanh lịch hơn với sợi vải t...,ARCTIC HUNTER,Thời trang nữ


Nhận xét:
- Cột `Description` còn nhiều dòng bắt đầu bằng các cụm từ không có ý nghĩa, lặp lại nhiều lần: "thông tin sản phẩm", "giới thiệu sản phẩm", "chi tiết sản phẩm"...
- Lựa chọn loại bỏ để thông tin giá trị hơn

In [33]:
def remove_boilerplate(text):
    if not isinstance(text, str) or len(text) == 0:
        return ""
    patterns = [
        r"^mô tả sản phẩm",
        r"^thông tin sản phẩm", 
        r"^chi tiết sản phẩm",
        r"^thông số kỹ thuật",
        r"^giới thiệu sản phẩm"
    ]
    combined_pattern = "|".join(patterns)
    text = re.sub(combined_pattern, "", text).strip()
    return text

# Áp dụng CHỈ cho cột description
df_cleaned['description'] = df_cleaned['description'].apply(remove_boilerplate)

In [34]:
#KIỂM TRA LẠI SAU XỬ LÝ CÓ CHUỖI RỖNG KHÔNG
def check_empty_strings(df):
    for col in df.columns:
        if df[col].dtype == "object":
            num_empty = (df[col].str.strip() == "").sum()
            print(f"Số chuỗi rỗng trong cột '{col}': {num_empty}")
check_empty_strings(df_cleaned)

Số chuỗi rỗng trong cột 'product_name': 0
Số chuỗi rỗng trong cột 'description': 0
Số chuỗi rỗng trong cột 'brand': 0
Số chuỗi rỗng trong cột 'category': 0


# 5. LỌC CÁC SẢN PHẨM KHÔNG ĐỦ ĐIỀU KIỆN
Nhận xét:    
- Một số sản phẩm có tên quá ngắn hoặc mô tả quá ít từ, hay tên chỉ là số, không đủ thông tin.  
- Hướng xử lý: loại bỏ các sản phẩm không đạt tiêu chuẩn tối thiểu (tên ≥2 từ, mô tả ≥5 từ, tên không phải số) để đảm bảo dữ liệu sạch và chất lượng hơn.


In [35]:
df_cleaned['name_word_count'] = df_cleaned['product_name'].str.split().str.len()
df_cleaned['desc_word_count'] = df_cleaned['description'].str.split().str.len()

df_cleaned = df_cleaned[
    (df_cleaned['name_word_count'] >= 2) &  # Tên ít nhất 2 từ
    (df_cleaned['desc_word_count'] >= 5)  &   # Mô tả ít nhất 5 từ
    (df_cleaned['product_name'].str.isnumeric() == False)  # Tên không phải là số
].copy()
print(f"Số dòng sau khi loại bỏ sản phẩm không đủ tiêu chuẩn: {len(df_cleaned)}")
print(f"Số dòng đã loại bỏ: {len(df) - len(df_cleaned)}")

Số dòng sau khi loại bỏ sản phẩm không đủ tiêu chuẩn: 7274
Số dòng đã loại bỏ: 15


# Đổi tên cột 'name' thành 'product_name' để khớp với model_training.ipynb
df_cleaned = df_cleaned.rename(columns={'name': 'product_name'})

# Chọn các cột cần lưu
columns_to_save = ['product_name', 'description', 'brand', 'category', 'text', 'source']
columns_to_save = [col for col in columns_to_save if col in df_cleaned.columns]

# Lưu vào folder processed
output_path = '../data/processed/data_cleaned.csv'
df_cleaned[columns_to_save].to_csv(output_path, index=False, encoding='utf-8')

print(f"\n✅ Đã lưu dữ liệu vào: {output_path}")
print(f"📊 Số rows: {len(df_cleaned)}")
print(f"📊 Số columns: {len(columns_to_save)}")

In [36]:
columns_to_save = ['product_name', 'description', 'brand', 'category']
df_cleaned[columns_to_save].to_csv('data_cleaned.csv', index=False, encoding='utf-8')